<a href="https://colab.research.google.com/github/habibi-bd/Generative-AI/blob/main/chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install beautifulsoup4
# !pip install pypdf
# !pip install sentence-transformers faiss-cpu langchain
!pip install chromadb


In [ ]:
!pip install -r requirements.txt

In [ ]:
import re

def clean_text(text):

    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text




In [ ]:
web_url="https://lilianweng.github.io/posts/2023-06-23-agent/"

In [ ]:
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from bs4 import BeautifulSoup

**TEXT DATA**

In [ ]:
text_loader= TextLoader("/content/speech.txt")
text=text_loader.load()
text=text[0].page_content

**WEB DATA**

In [ ]:
from bs4 import BeautifulSoup, SoupStrainer

In [ ]:
web_loader = WebBaseLoader(
    web_path=web_url,
    bs_kwargs=dict(parse_only=SoupStrainer(class_=("post-title", "post-content", "post-header")))
)

web_data = web_loader.load()


# PDF DATA

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader=PyPDFLoader('/content/Hands On Machine Learning with Scikit Learn and TensorFlow.pdf')
pdf_data=pdf_loader.load()


**SPLITING INTO CHUNK**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [ ]:
documents = text_splitter.split_documents(pdf_data)

In [ ]:
# len(documents)

In [ ]:
for i ,doc in enumerate(documents[:5]):
  print(f"Document {i+1} has {len(doc.page_content)} characters")
for i, doc in enumerate(documents[:5]):
  print(f"\n Chunk {i+1}\n")
  print(doc.page_content)

In [ ]:
from langchain.vectorstores import FAISS
import faiss
from sentence_transformers import SentenceTransformer


**LOAD MODEL**

In [ ]:
model = SentenceTransformer('all-mpnet-base-v2')


In [ ]:
pdf_loader=PyPDFLoader('/content/Attention all you need 1706.03762v7.pdf')
pdf_data=pdf_loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents = text_splitter.split_documents(pdf_data)


texts=[doc.page_content for doc in documents]
embedding=model.encode(texts,show_progress_bar=True)

In [ ]:
index= faiss.IndexFlatL2(embedding.shape[1])
faiss.write_index(index, "faiss_index.index")

In [ ]:
query="FAISS"
query_embedding=model.encode([query])

# print(query_embedding)

In [ ]:

D,I=index.search(query_embedding, k=5)


In [ ]:
for idx in I[0]:
  print(f"Mathced Documents {texts[idx]}")

In [ ]:
query = "What is attention"
query_embedding = model.encode([web_data[0].page_content])

In [ ]:
D,I=index.search(query_embedding, k=2)
# for x in I[0]:
#   print(f"Matched Documents {texts[x]}")
I[0]

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')


In [ ]:
pdf_loader=PyPDFLoader('/content/Attention all you need 1706.03762v7.pdf')

pdf_data=pdf_loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

documents = text_splitter.split_documents(pdf_data)
texts= [doc.page_content for doc in documents]
# max_length=512

# filtered_text=[text[:max_length] for text in texts]
embeddings=model.encode(texts, show_progress_bar=True)

In [ ]:
query = "dlfajdl"
query_embedding=model.encode([query])
D,I=index.search(query_embedding, k=5)

In [ ]:
for x in I[0]:
  print(texts[x])

**CHROMADB
CHROMADB
CHROMADB
CHROMADB
CHROMADB
**

In [ ]:
"""
🔍 What Does ChromaDB Do?

ChromaDB is a vector database that helps you store, index, and retrieve high-dimensional vectors (embeddings) efficiently. It's particularly useful when you're dealing with tasks like semantic search, recommendation systems, question answering, and similarity-based retrieval.

🚀 Why ChromaDB?
✅ Efficient Storage: Keeps embeddings (vectors) in a structured way with metadata.

✅ Fast Retrieval: Uses indexing techniques to perform quick similarity searches.

✅ Scalability: Handles large datasets and complex queries effectively.

✅ Metadata Support: You can store text chunks with additional metadata for better filtering.

📌 What It Actually Does
Embeddings Storage

When you generate embeddings from your text chunks using a model like SentenceTransformer, ChromaDB stores these embeddings along with their text content and metadata.

Think of it as saving text in a special way where meaning is captured by a vector (a list of numbers).

Indexing

ChromaDB builds efficient indices for those embeddings. This helps in making search operations much faster by organizing the vectors in a way that allows quick comparisons.

It usually uses techniques like FAISS (Facebook AI Similarity Search) under the hood for fast retrieval.

Retrieval

When you provide a query, it is converted into an embedding by your model.

ChromaDB compares this query embedding against all stored embeddings and returns the most similar ones (based on cosine similarity or Euclidean distance).

This is what allows you to ask questions like "What is attention?" and find relevant text chunks.

Metadata Handling (Bonus)

You can store extra information about your texts (e.g., page numbers, sections, tags) and filter results based on those.


"""

In [ ]:
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import chromadb

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
pdf_loader = PyPDFLoader('/content/Attention all you need 1706.03762v7.pdf')
# pdf_loader= TextLoader("/content/nlp.txt")

pdf_data = pdf_loader.load()

In [ ]:
text_spliter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_spliter.split_documents(pdf_data)
texts=[doc.page_content for doc in documents if doc.page_content.strip()!=""]
embeding=model.encode(texts, show_progress_bar=True)

In [ ]:
client=chromadb.Client()
client.delete_collection("my_nlpd")


In [ ]:
client=chromadb.Client()
collection=client.create_collection("my_nlpd")
collection.add(
    documents=texts,
    embeddings=embeding,
    ids=[str(i) for i in range(len(texts))]
)

In [ ]:
query="why embedding require"
query_embedding=model.encode([query])
result=collection.query(query_embeddings=query_embedding, n_results=5)


In [ ]:
for res in result["documents"]:
  print(res)